# Cat & dog image classification

Выполнил: студент группы А-14м-16 Мигаль Иван.

Описание:

Решение задачи классификации образов (кошек и собак) с помощью keras. 

Для начала подключим необходимые модули и выведем их версии, в том числе версию python3. 
Также выведем абсолютный путь к папке с python3 (для разработчиков).

In [12]:
%matplotlib inline
import keras
import os
import matplotlib
import urllib.request
from matplotlib import pyplot as plt
import sys
import getpass

usrname = getpass.getuser()

print('Версия python3:', sys.version)
print('Версия keras:', keras.__version__)
print('Версия matplotlib:', matplotlib.__version__)
print('Версия urlib.request', urllib.request.__version__)
print('Абсолютный путь к папке python3:', sys.executable)
print('Имя пользователя', usrname)

Версия python3: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Версия keras: 2.0.9
Версия matplotlib: 1.5.3
Версия urlib.request 3.5
Абсолютный путь к папке python3: /home/ivmig/anaconda3/bin/python
Имя пользователя ivmig


In [11]:
if not os.path.exists('/data_no_split/'):
    os.makedirs('/home/'+usrname+'/data_no_split/')

In [ ]:
%%time

url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'
urllib.request.urlretrieve(url, '/home/'+usrname+'/data_no_split/cat_dog_images.zip')  

In [ ]:
%%time

url = 'https://drive.google.com/uc?export=download&confirm=1laB&id=0Bz7KyqmuGsilT0J5dmRCM0ROVHc'
urllib.request.urlretrieve(url, '/home/'+usrname+'/vgg16_weights.h5') 